In [89]:
from credentials import *
import requests
import json, csv
import warnings

from getFEC import *

# Issues

### Schedule A

Schedule A shows where individual donations go to. Some go to individual candidate fundraising committees or committees affiliated with the candidate, in which case, the committee id appears to return a party id.

Schedule A may also go to a PAC (Political Action Committee) such as "ACTBLUE", committee id `"C00401224"`, which will have null results when trying to directly search the party id details. This is because the PAC gives to a number of campaigns.

### Schedule B

To track down a PAC giving, the committee id of the PAC must be input into the schedule_b api query, which returns a list of other committees the money is given to. Those committee id's could be used to get the party ID from.



## Psudocode

```python
infer_partyid(comm_id)
    #if regular comm this works
    try:
        party = get_party_id(comm_id)
        
    #fail, fails but would be good to design a better catch
    except:
       schedule_b_search:
           a = comm_id #first committee
           b = comm_id #second committee
           if a == b:
               a = partyid
               return a
               
           elif a != b:
               a = partyid
               create new flag that there are multiple party id's, warning
               return a
           else:
               pass
           
```


In [111]:
def get_party_id(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/candidates/?sort=name&page=1&api_key={}&per_page=100".format(committee_id, api_key)
    url = stem+end

    data = get_url(url)['results']
    if len(data) <= 0:
        warnings.warn('WARNING: get_party_id search by committee id returns no results, try another method')
        return ("UNKNOWN PARTY ID")
    else:
        party_id = data[0]['party_full']
        return party_id

In [113]:
get_party_id("C00000935")
get_party_id("C00464297")

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: WARNING: get_party_id search by committee id returns no results, try another method


'REPUBLICAN PARTY'

In [115]:
def get_committee_details(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/"
    end = "{}/?sort=name&api_key={}&per_page=100&page=1".format(committee_id, api_key)
    url = stem+end
    
    data = get_url(url)['results']
    party_id = data[0]['party_full']
    candidate_ids = data[0]['candidate_ids']
    #office = data[0]['office_full']
    #name = data[0]['name']
    #return data[0]['party']
    return party_id, candidate_ids

get_committee_details("C00000935")

('DEMOCRATIC PARTY', [])

In [45]:
#id from actblue schedule b
#gets result from get_committee details
#fails get_party_id
get_committee_details("C00000935")

('DEM', [])

In [116]:
get_committee_details("C00005561")

('DEMOCRATIC PARTY', [])

In [47]:
#does not fail get_party_id
#get_committee_details has candidate_id
#get_committee_details has no party id
get_committee_details("C00464297")

(None, ['S0NH00235'])

In [48]:
#PAC
#fails get party id
#returns none with get comm details
#no candidate_id
get_committee_details("C00401224")

(None, [])

In [58]:
party_results = get_committee_details("C00401224")
party_results[1]
len(party_results[1])

0

In [118]:
party_id = get_party_id("C00464297")

In [144]:
def get_schedule_b_receipts(committee_id):
    stem = "https://api.open.fec.gov/v1/committee/{}/schedules/".format(committee_id)
    end = "schedule_b/by_recipient_id/?per_page=20&api_key={}&page=1".format(api_key)
    url = stem+end
    
    data = get_url(url)['results']
    if len(data) <= 0:
        warnings.warn('WARNING: get_schedule_b_receipts search by committee id returns no results, try another method')
        return None
    elif len(data) > 0 and len(data) < 3:
        return data[0]['recipient_id']
    elif len(data) > 0 and len(data) >= 3:
        #receipts
        r1 = data[0]['recipient_id']
        r2 = data[1]['recipient_id']
        r3 = data[2]['recipient_id']        
        return [r1, r2, r3]
    else:
        pass

x = get_schedule_b_receipts("C00401224")
type(x)

list

In [190]:
def find_schedule_b_results(committee_ids):
    if type(committee_ids) is list:
        cids = committee_ids
    else:
        cids = [committee_ids]
    assert len(cids) > 0
    
    if len(cids) == 1:
        #print(cids)
        party_id = get_committee_details(cids[0])[0]
        return party_id
    else:
        
        pid_1 = get_committee_details(cids[0])[0]
        pid_2 = get_committee_details(cids[1])[0]
        pid_3 = get_committee_details(cids[2])[0]
        #print(pid_1, pid_2, pid_3)
        if (pid_1 == pid_2 == pid_3) is True:
            party_id = pid_1
            return party_id
        else:
            return "UNCLEAR Schedule B Party ID"
        
#find_schedule_b_results(('C00000935', 'C00005173', 'C00005561'))

#find_schedule_b_results('C00000935')
#find_schedule_b_results(x)

In [186]:
test = get_schedule_b_receipts("C00571380")

In [187]:
find_schedule_b_results(test)

REPUBLICAN PARTY REPUBLICAN PARTY REPUBLICAN PARTY


'REPUBLICAN PARTY'

In [191]:
#party_results = get_committee_details("C00401224")
#party_results = get_committee_details("C00000935")
#party_results = get_committee_details("C00464297")

def search_party_id(committee_id):
    
    party_results = get_committee_details(committee_id)

    if party_results[0] is None and len(party_results[1]) == 0:
        #print("no party results, further_tests_needed", party_results)
        #print("[*] conducting schedule b search...")
        schedule_b_receipts = get_schedule_b_receipts(committee_id)
        party_id = find_schedule_b_results(schedule_b_receipts)
        return party_id
    
    elif party_results[0] is None and len(party_results[1]) > 0:
        #print("candidate id exists but no party results", party_results)
        #try get_party_id search
        party_id = get_party_id(committee_id)
        return party_id
    
    elif party_results[0] is not None:
        #print("party id found in committee details: ", party_results)
        party_id = party_results[0]
        return party_id
    
    else:
        #print("pass, unknown results", party_results)
        party_id = "UNKNOWN D"
        return party_id
    
test_ids = ["C00401224", "C00000935", "C00464297", "C00571380"]
#test_manual_inf = ["dem pac", "dem comm", "rep can", "rep pac"]
for cid  in test_ids:
    party_id = search_party_id(cid)
    print(party_id)

DEMOCRATIC PARTY
DEMOCRATIC PARTY
REPUBLICAN PARTY
REPUBLICAN PARTY
